In [ ]:
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
with open("./feature/APE_data.npy", "rb") as f:
    X = np.load(f)
    sell_y = np.load(f)
    buy_y = np.load(f)

In [ ]:
X.shape, sell_y.shape, buy_y.shape

In [ ]:
# create your datset and dataloader
sell_dataset = TensorDataset(torch.Tensor(X), torch.Tensor(sell_y)) 
sell_train_dataset, sell_val_dataset = random_split(sell_dataset, [18474, 23093 - 18474])
sell_dataloader = DataLoader(sell_train_dataset, batch_size=128, shuffle=True) 
sell_val_loader = DataLoader(sell_val_dataset) 

buy_dataset = TensorDataset(torch.Tensor(X), torch.Tensor(buy_y)) 
buy_train_dataset, buy_val_dataset = random_split(buy_dataset, [18474, 23093 - 18474])
buy_dataloader = DataLoader(buy_train_dataset, batch_size=128, shuffle=True)
buy_val_loader = DataLoader(buy_val_dataset) 

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(22, 16),
            nn.Linear(16, 11),
            nn.ReLU(),
            nn.Linear(11, 11),
        )

    def forward(self, x):
        v = self.network(x)
        return v

In [ ]:
model_name = "APE_BUY"
buy_model = SimpleNN()
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(buy_model.parameters(), lr=1e-5)

best_loss = 10000
for epoch_i in range(1000):
    train_loss_array = []
    val_loss_array = []
    # train
    for data_X, data_y in buy_dataloader:
        y_hat = buy_model(data_X)
        loss = loss_fn(y_hat, data_y)
        train_loss_array.append(loss.item())
        loss.backward()    
        optimizer.step()
        optimizer.zero_grad()

    # eval
    with torch.no_grad():
        for x_val, y_val in buy_val_loader:
            # x_val = x_val.to(device)
            # y_val = y_val.to(device)
            buy_model.eval()
            yhat = buy_model(x_val)
            val_loss = loss_fn(y_val, yhat)
            val_loss_array.append(val_loss.item())
    
    if epoch_i%10 == 0:
        print (f"Epoch {epoch_i}: Training Loss: {np.mean(train_loss_array)}; Test Loss: {np.mean(val_loss_array)}")
    if epoch_i%100 == 99:
        torch.save(buy_model, f"./model/checkpoint/model_ckpt_{epoch_i}.save")
        if np.mean(val_loss_array) < best_loss:
            torch.save(buy_model, f"./model/{model_name}_model.save")
            best_loss = np.mean(val_loss_array)

In [ ]:
model_name = "APE_SELL"
sell_model = SimpleNN()
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(sell_model.parameters(), lr=1e-5)

best_loss = 10000
for epoch_i in range(1000):
    train_loss_array = []
    val_loss_array = []
    # train
    for data_X, data_y in sell_dataloader:
        y_hat = sell_model(data_X)
        loss = loss_fn(y_hat, data_y)
        train_loss_array.append(loss.item())
        loss.backward()    
        optimizer.step()
        optimizer.zero_grad()

    # eval
    with torch.no_grad():
        for x_val, y_val in sell_val_loader:
            # x_val = x_val.to(device)
            # y_val = y_val.to(device)
            sell_model.eval()
            yhat = sell_model(x_val)
            val_loss = loss_fn(y_val, yhat)
            val_loss_array.append(val_loss.item())
    
    if epoch_i%10 == 0:
        print (f"Epoch {epoch_i}: Training Loss: {np.mean(train_loss_array)}; Test Loss: {np.mean(val_loss_array)}")
    if epoch_i%100 == 99:
        torch.save(buy_model, f"./model/checkpoint/model_ckpt_{epoch_i}.save")
        if np.mean(val_loss_array) < best_loss:
            torch.save(sell_model, f"./model/{model_name}_model.save")
            best_loss = np.mean(val_loss_array)

#### Test

In [ ]:
test_data_X, test_data_y = buy_dataset.__getitem__(1000)
buy_model(test_data_X), test_data_y